In [ ]:
import pandas as pd

# Csak O
def load_and_process_measurements(csv_path: str) -> pd.DataFrame:
    df = pd.read_csv(csv_path)

    df['label'] = -1
    current_label = -1
    in_measurement = False

    for idx, row in df.iterrows():
        if (row == 'O').any():
            current_label += 1
            in_measurement = True
        if in_measurement:
            df.at[idx, 'label'] = current_label
        else:
            df.at[idx, 'label'] = -1

    df = df[~df.apply(
        lambda row: row.astype(str).eq('X').any() or row.astype(str).eq('O').any(), axis=1)]

    df = df[df['label'] != -1]
    df.reset_index(inplace=True, drop=True)

    df = df.astype(float)

    df['time'] = df['Timestamp'] - df['Timestamp'].iloc[0]

    df['relative_time'] = df.groupby('label')['Timestamp'].transform(lambda x: x - x.iloc[0])

    df = df.drop(columns=['Timestamp'])

    return df


In [ ]:
import pandas as pd

# X O
def load_and_process_measurements2(csv_path: str) -> pd.DataFrame:
    df = pd.read_csv(csv_path)

    df['label'] = -1
    current_label = -1
    in_measurement = False

    for idx, row in df.iterrows():
        if (row == 'X').any():
            current_label += 1
            in_measurement = True
            df.at[idx, 'label'] = current_label
        elif (row == 'O').any():
            df.at[idx, 'label'] = current_label
            in_measurement = False
        elif in_measurement:
            df.at[idx, 'label'] = current_label
        else:
            df.at[idx, 'label'] = -1

    df = df[~df.apply(
        lambda row: row.astype(str).eq('X').any() or row.astype(str).eq('O').any(), axis=1)]

    df = df[df['label'] != -1]
    df.reset_index(inplace=True, drop=True)

    df = df.astype(float)

    df['time'] = df['Timestamp'] - df['Timestamp'].iloc[0]

    df['relative_time'] = df.groupby('label')['Timestamp'].transform(lambda x: x - x.iloc[0])

    df = df.drop(columns=['Timestamp'])

    return df


In [2]:
correct = load_and_process_measurements(r'correct_evezesek.csv')
print('Correct circles length:', correct.shape[0], 'Number of measurements:', correct['label'].nunique(), 'Avg measurement length:', correct.shape[0] / correct['label'].nunique())
print('Longest measuremet:', correct['label'].value_counts().max(), 'Shortest:', correct['label'].value_counts().min())
correct.head()

Correct circles length: 1941 Number of measurements: 44 Avg measurement length: 44.11363636363637
Longest measuremet: 150 Shortest: 24


,linear_accelerationX,linear_accelerationY,linear_accelerationZ,magX,magY,magZ,gyroX,gyroY,gyroZ,accX,accY,accZ,label,time,relative_time
0,-0.127381,0.314876,-0.094141,30.834961,-1.953125,-22.460938,5.187988,20.568848,8.117676,0.844116,0.266357,-0.068115,0.0,0.000000,0.000000
1,-0.238485,-0.026236,0.336964,29.370117,-2.282715,-22.265625,37.292480,16.723633,8.056641,0.729004,-0.075195,0.368652,0.0,0.044548,0.044548
2,0.174476,0.048510,0.285294,29.760742,-1.806641,-25.646973,31.921387,8.850098,3.051758,1.144897,0.000366,0.321777,0.0,0.089834,0.089834
3,0.226336,0.196694,0.497033,28.637695,-1.916504,-24.841309,-17.944336,8.483887,4.150391,1.200562,0.151855,0.541870,0.0,0.134616,0.134616
4,0.129674,-0.113292,0.566852,29.809570,-1.855469,-24.377441,-57.006836,-0.488281,1.770020,1.106079,-0.160034,0.621216,0.0,0.179619,0.179619


In [6]:
incorrect = load_and_process_measurements2(r'correct_circles.csv')
print('Incorrect circles length:', incorrect.shape[0], 'Number of measurements:', incorrect['label'].nunique(), 'Avg measurement length:', incorrect.shape[0] / incorrect['label'].nunique())
print('Longest measuremet:', incorrect['label'].value_counts().max(), 'Shortest:', incorrect['label'].value_counts().min())
incorrect.head()

Incorrect circles length: 7108 Number of measurements: 183 Avg measurement length: 38.84153005464481
Longest measuremet: 65 Shortest: 23


,linear_accelerationX,linear_accelerationY,linear_accelerationZ,magX,magY,magZ,gyroX,gyroY,gyroZ,accX,accY,accZ,label,time,relative_time
0,-0.045276,-0.086649,0.045080,3.186035,32.385254,-98.010254,-2.563477,4.272461,-0.183105,-0.014771,-0.000977,0.972534,0.0,0.000000,0.000000
1,-0.067938,-0.105986,0.042895,3.540039,32.666016,-95.935059,-1.525879,5.065918,-3.112793,-0.038574,-0.022095,0.971069,0.0,0.001999,0.001999
2,-0.044365,-0.079023,0.043506,3.540039,32.702637,-96.582031,-0.488281,4.882812,-1.525879,-0.015747,0.003540,0.972412,0.0,0.059856,0.059856
3,-0.045673,-0.076636,0.048670,3.308105,32.617188,-96.887207,0.854492,5.065918,-0.244141,-0.017822,0.004639,0.978394,0.0,0.120137,0.120137
4,-0.039875,-0.076690,0.041502,3.430176,31.872559,-96.606445,1.464844,5.737305,-3.967285,-0.012695,0.003296,0.971924,0.0,0.239192,0.239192


In [7]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

def prepare_sequence_data(df: pd.DataFrame, label_value: int, max_len: int = None):
    feature_cols = df.columns.difference(['label', 'time'])
    sequences = []
    labels = []
    
    grouped = df.groupby('label')

    for _, group in grouped:
        seq = group[feature_cols].values
        sequences.append(seq)
        labels.append(label_value)

    if max_len is None:
        max_len = max(len(seq) for seq in sequences)

    sequences_padded = pad_sequences(sequences, maxlen=max_len, padding='post', dtype='float32')
    labels = np.array(labels)

    return sequences_padded, labels


In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D, Dense, Dropout, Masking

X_correct, y_correct = prepare_sequence_data(correct, label_value=1)
X_incorrect, y_incorrect = prepare_sequence_data(incorrect, label_value=0, max_len=X_correct.shape[1])

X = np.concatenate([X_correct, X_incorrect], axis=0)
y = np.concatenate([y_correct, y_incorrect], axis=0)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = Sequential([
    Masking(mask_value=0., input_shape=(X.shape[1], X.shape[2])),
    Conv1D(64, kernel_size=5, activation='relu'),
    GlobalMaxPooling1D(),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking_1 (Masking)         (None, 150, 13)           0         
                                                                 
 conv1d_1 (Conv1D)           (None, 146, 64)           4224      
                                                                 
 global_max_pooling1d_1 (Gl  (None, 64)                0         
 obalMaxPooling1D)                                               
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 1)                 33        
                                                      

In [12]:
from tensorflow.keras.callbacks import ModelCheckpoint

ch = ModelCheckpoint(
    'best_model.h5',
    monitor='val_accuracy',
    verbose=0,
    save_best_only=True,
    mode='auto'
)

In [13]:
model.fit(X_train, y_train, epochs=20, batch_size=8, validation_split=0.2, callbacks=[ch])

loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc:.2f}")

Epoch 1/20
18/18 [==============================] - 1s 21ms/step - loss: 6.6339 - accuracy: 0.7500 - val_loss: 7.4268e-06 - val_accuracy: 1.0000
Epoch 2/20
18/18 [==============================] - 0s 7ms/step - loss: 0.6263 - accuracy: 0.9514 - val_loss: 6.6149e-08 - val_accuracy: 1.0000
Epoch 3/20
 1/18 [>.............................] - ETA: 0s - loss: 0.0576 - accuracy: 1.0000

c:\Users\asus\.conda\envs\block_c\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


18/18 [==============================] - 0s 7ms/step - loss: 0.2093 - accuracy: 0.9653 - val_loss: 1.7708e-09 - val_accuracy: 1.0000
Epoch 4/20
18/18 [==============================] - 0s 7ms/step - loss: 0.1316 - accuracy: 0.9792 - val_loss: 1.2677e-10 - val_accuracy: 1.0000
Epoch 5/20
18/18 [==============================] - 0s 8ms/step - loss: 0.1289 - accuracy: 0.9861 - val_loss: 9.3121e-11 - val_accuracy: 1.0000
Epoch 6/20
18/18 [==============================] - 0s 8ms/step - loss: 0.1187 - accuracy: 0.9861 - val_loss: 2.5483e-11 - val_accuracy: 1.0000
Epoch 7/20
18/18 [==============================] - 0s 7ms/step - loss: 0.0057 - accuracy: 0.9931 - val_loss: 5.7924e-11 - val_accuracy: 1.0000
Epoch 8/20
18/18 [==============================] - 0s 7ms/step - loss: 0.0158 - accuracy: 0.9931 - val_loss: 7.2390e-11 - val_accuracy: 1.0000
Epoch 9/20
18/18 [==============================] - 0s 7ms/step - loss: 0.0639 - accuracy: 0.9861 - val_loss: 8.5067e-11 - val_accuracy: 1.0000
Epo

In [14]:
from tensorflow.keras.models import load_model

best_model = load_model('best_model.h5')

loss, acc = best_model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc:.2f}")

2/2 [==============================] - 0s 5ms/step - loss: 0.0459 - accuracy: 0.9783
Test Accuracy: 0.98
